In [128]:
import numpy as np
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
URM_path="../data/interactions_and_impressions.csv"
URM_path2="../data/interactions_and_impressions_v4.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                skiprows=1,
                                header=None,
                                dtype={0: int, 1: int, 2: str, 3: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "others","Data"]
#URM_all_dataframe = URM_all_dataframe.sort_values(by=['UserID', 'ItemID', 'Data'])


In [129]:

URM_all_dataframe2 = pd.read_csv(URM_path2)
URM_all_dataframe2.columns = ["UserID", "ItemID","Data"]
URM_all_dataframe2 = URM_all_dataframe2[URM_all_dataframe2['Data'] != 0.01]
#URM_all_dataframe2 = URM_all_dataframe2.sort_values(by=['UserID', 'ItemID', 'Data'])


In [130]:
'''
all_users=list(URM_all_dataframe["UserID"].unique())
np.random.shuffle(all_users)


import random
train_percentage=0.8
user_for_training=random.sample(all_users, round(len(all_users)*train_percentage))
data_train2=URM_all_dataframe2[URM_all_dataframe2["UserID"].isin(user_for_training)]
data_train=URM_all_dataframe[URM_all_dataframe["UserID"].isin(user_for_training)]
data_valid=URM_all_dataframe[~URM_all_dataframe["UserID"].isin(user_for_training)]
data_valid2=URM_all_dataframe2[~URM_all_dataframe2["UserID"].isin(user_for_training)]
'''

'\nall_users=list(URM_all_dataframe["UserID"].unique())\nnp.random.shuffle(all_users)\n\n\nimport random\ntrain_percentage=0.8\nuser_for_training=random.sample(all_users, round(len(all_users)*train_percentage))\ndata_train2=URM_all_dataframe2[URM_all_dataframe2["UserID"].isin(user_for_training)]\ndata_train=URM_all_dataframe[URM_all_dataframe["UserID"].isin(user_for_training)]\ndata_valid=URM_all_dataframe[~URM_all_dataframe["UserID"].isin(user_for_training)]\ndata_valid2=URM_all_dataframe2[~URM_all_dataframe2["UserID"].isin(user_for_training)]\n'

In [131]:
all_users_items=list(zip(URM_all_dataframe['UserID'],URM_all_dataframe['ItemID']))
np.random.shuffle(all_users_items)


import random
train_percentage=0.8
user_for_training=random.sample(all_users_items, round(len(all_users_items)*train_percentage))
user_for_training=pd.DataFrame(user_for_training,columns=["UserID","ItemID"])
data_train2 = pd.merge(URM_all_dataframe2,user_for_training.drop_duplicates())
data_train = pd.merge(URM_all_dataframe,user_for_training.drop_duplicates())
data_valid2=pd.concat([URM_all_dataframe2,data_train2]).drop_duplicates(keep=False)
data_valid=pd.concat([URM_all_dataframe,data_train]).drop_duplicates(keep=False)

In [132]:
len(URM_all_dataframe2)

5826506

In [133]:
len(URM_all_dataframe)

5826506

In [134]:
import numpy as np
import scipy.sparse as sps


URM_train = sps.csr_matrix((data_train["Data"].values,
                          (data_train["UserID"].values, data_train["ItemID"].values)))
URM_train2 = sps.csr_matrix((data_train2["Data"].values,
                          (data_train2["UserID"].values, data_train2["ItemID"].values)))
URM_valid = sps.csr_matrix((data_valid["Data"].values,
                             (data_valid["UserID"].values, data_valid["ItemID"].values)))
URM_valid2 = sps.csr_matrix((data_valid2["Data"].values,
                             (data_valid2["UserID"].values, data_valid2["ItemID"].values)))
URM_train,URM_train2 ,URM_valid,URM_valid2

(<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1311824 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1311824 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 239760 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 239413 stored elements in Compressed Sparse Row format>)

In [141]:
len(data_train)

5577989

In [142]:
len(data_train2)


5577989

In [143]:
len(data_valid)


241681

In [144]:
len(data_valid2)


240931

In [151]:
diff = data_train[data_train.apply(tuple,1).isin(data_train2.apply(tuple,1))]


In [152]:
len(diff)

2052496

In [135]:
from Utils.Evaluator import EvaluatorHoldout

evaluator_validation=EvaluatorHoldout(URM_valid,cutoff_list=[10])
evaluator_validation2=EvaluatorHoldout(URM_valid2,cutoff_list=[10])


EvaluatorHoldout: Ignoring 8129 (19.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2504 ( 6.0%) Users that have less than 1 test interactions


In [136]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

ItemUserHybridKNNRecommender: URM Detected 1 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 5070.49 column/sec. Elapsed time 4.83 sec
Similarity column 41629 (100.0%), 1879.87 column/sec. Elapsed time 22.14 sec
[[-0.06986804 -0.2586211  -0.20992817 ... -0.30984303 -0.1801217
  -0.29034597]
 [ 0.2912927   0.16658685  0.5597415  ... -0.11639152 -0.19299506
  -0.17611946]
 [-0.01944898  0.19089526  0.2274145  ... -0.07644556 -0.19516653
  -0.23870586]
 ...
 [-0.02030004 -0.24388649 -0.15284157 ... -0.30984303  0.03551585
  -0.12604082]
 [-0.30984303 -0.27769318 -0.30984303 ... -0.2732101  -0.30984303
  -0.30984303]
 [ 0.16933116  0.13876402  0.31093055 ... -0.24366897  0.555451
  -0.10046581]]
[[-0.04634393 -0.03410903  0.00328933 ... -0.04634393 -0.00539218
  -0.04634393]
 [-0.03950318 -0.04634393 -0.00300879 ... -0.03772

KeyboardInterrupt: 

In [ ]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

In [ ]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df